# Description

TODO

# Modules

In [1]:
# import pandas as pd
import numpy as np
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
import seaborn as sns
# from sklearn.preprocessing import minmax_scale

from clustermatch import conf
from clustermatch.coef import cm
from clustermatch.methods import mic, distcorr

ModuleNotFoundError: No module named 'minepy'

# Settings

In [ ]:
N_REPS = 100
N_SAMPLES_LIST = [10, 50, 100, 500]

# Paths

In [2]:
assert (
    conf.MANUSCRIPT["BASE_DIR"] is not None and conf.MANUSCRIPT["BASE_DIR"].exists()
), "Manuscript dir not set"

In [3]:
OUTPUT_FIGURE_DIR = conf.MANUSCRIPT["FIGURES_DIR"] / "intro"
OUTPUT_FIGURE_DIR.mkdir(parents=True, exist_ok=True)
display(OUTPUT_FIGURE_DIR)

PosixPath('/opt/manuscript/content/images/relationship_types')

# Run

In [ ]:
cm_times = []
mic_times = []
distcorr_times = []

In [ ]:
for i, n_samples in enumerate(N_SAMPLES_LIST):
        x = np.random.rand(size=n_samples)
        y = np.random.rand(size=n_samples)
        
        avl_times[i] = timeit(avl, number=n_times)